# 🧬 Analyse Dhātu Multi-Hypothèses Intensive

**PaniniFS Research - Processing Intensif GPU pour Hypothèses Multiples**

[![GitHub](https://img.shields.io/badge/GitHub-PaniniFS--Research-blue)](https://github.com/stephanedenis/PaniniFS-Research)

## 🎯 Objectifs du Traitement Intensif

1. **Analyse Multi-Hypothèses** : Pour chaque document, tester multiples interprétations dhātu
2. **Multi-Langues** : Français, Anglais, Sanskrit translittéré
3. **Variantes Aspectuelles** : Chaque dhātu testé sous différents aspects temporels
4. **Combinaisons Complexes** : Patterns dhātu composés et interactions
5. **GPU Long Processing** : Optimisé pour sessions de plusieurs heures

---

⚠️ **Configuration Required**: Runtime → Change runtime type → GPU (T4 ou V100)

In [ ]:
# 🔧 Configuration GPU et Environnement
import torch
import numpy as np
import json
import time
import re
from datetime import datetime
from collections import defaultdict, Counter
from typing import Dict, List, Any, Tuple
import requests
from pathlib import Path
import itertools
from dataclasses import dataclass

# Vérification GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔥 Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"   CUDA: {torch.version.cuda}")

# Session info
session_id = f"intensive_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
print(f"\n🧬 Session ID: {session_id}")
print(f"⏱️ Start Time: {datetime.now()}")

In [ ]:
# 🔄 Synchronisation GitHub
!git clone https://github.com/stephanedenis/PaniniFS-Research.git
%cd PaniniFS-Research

# Configuration Git pour commits
!git config user.email "research@paninifs.org"
!git config user.name "PaniniFS-Colab-Intensive"

print("📂 Repository synchronized")
!ls -la data/*.json | head -5

In [ ]:
# 📚 Chargement Corpus Multi-Sources

def load_all_corpus():
    """Charge tous les corpus disponibles"""
    
    corpus_files = [
        'data/corpus_scientifique.json',
        'data/corpus_multilingue_dev.json',
        'data/corpus_prescolaire.json'
    ]
    
    all_documents = []
    load_stats = {}
    
    for file_path in corpus_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # Handle different formats
            if isinstance(data, list):
                documents = data
            elif isinstance(data, dict) and 'documents' in data:
                documents = data['documents']
            else:
                print(f"⚠️ Format non reconnu: {file_path}")
                continue
            
            # Standardize document format
            for i, doc in enumerate(documents):
                if isinstance(doc, dict):
                    # Ensure required fields
                    doc['source_file'] = file_path
                    doc['doc_index'] = i
                    if 'id' not in doc:
                        doc['id'] = f"{Path(file_path).stem}_{i}"
                    if 'content' not in doc:
                        doc['content'] = doc.get('abstract', doc.get('text', ''))
                    if 'language' not in doc:
                        doc['language'] = 'en'  # default
                    
                    all_documents.append(doc)
            
            load_stats[file_path] = len(documents)
            print(f"✅ {file_path}: {len(documents)} documents")
            
        except Exception as e:
            print(f"❌ Erreur {file_path}: {e}")
            load_stats[file_path] = 0
    
    print(f"\n📊 CORPUS TOTAL: {len(all_documents)} documents")
    print(f"📈 Breakdown: {load_stats}")
    
    return all_documents, load_stats

# Load corpus
corpus_documents, corpus_stats = load_all_corpus()

# Sample document preview
if corpus_documents:
    print(f"\n📄 EXEMPLE DOCUMENT:")
    sample = corpus_documents[0]
    print(f"   ID: {sample.get('id', 'N/A')}")
    print(f"   Language: {sample.get('language', 'N/A')}")
    print(f"   Content: {sample.get('content', '')[:100]}...")

In [ ]:
# 🧬 Analyseur Dhātu Multi-Hypothèses GPU-Accéléré

@dataclass
class DhatuHypothesis:
    dhatu_root: str
    aspect: str  # 'present', 'past', 'future', 'perfect', 'continuous'
    intensity: float
    confidence: float
    language_variant: str
    contextual_factors: List[str]
    semantic_field: str

class IntensiveDhatuAnalyzer:
    """Analyseur intensif multi-hypothèses pour dhātu"""
    
    def __init__(self):
        self.device = device
        print(f"🧬 Analyseur intensif initialisé sur {self.device}")
        
        # Dhātu roots avec aspects multiples
        self.dhatu_database = {
            'RELATE': {
                'root': '√bandh',
                'patterns': {
                    'en': [r'connect', r'link', r'bind', r'relate', r'join', r'unite', r'associate'],
                    'fr': [r'relier', r'connecter', r'lier', r'unir', r'associer', r'joindre'],
                    'sa': [r'bandh', r'yuj', r'sam.*gam']
                },
                'aspects': ['present', 'continuous', 'perfect'],
                'semantic_fields': ['connection', 'relationship', 'unity']
            },
            'EXIST': {
                'root': '√as',
                'patterns': {
                    'en': [r'\bis\b', r'\bare\b', r'exist', r'being', r'there.*is', r'\bbe\b'],
                    'fr': [r'être', r'\best\b', r'existe', r'il.*y.*a', r'se.*trouve'],
                    'sa': [r'\bas\b', r'bhū', r'sat']
                },
                'aspects': ['present', 'continuous', 'eternal'],
                'semantic_fields': ['existence', 'being', 'reality']
            },
            'CAUSE': {
                'root': '√kṛ',
                'patterns': {
                    'en': [r'cause', r'make', r'create', r'produce', r'generate', r'effect'],
                    'fr': [r'causer', r'faire', r'créer', r'produire', r'générer', r'effectuer'],
                    'sa': [r'kṛ', r'jan', r'utpād']
                },
                'aspects': ['present', 'past', 'future', 'causal'],
                'semantic_fields': ['causation', 'creation', 'action']
            },
            'EVAL': {
                'root': '√man',
                'patterns': {
                    'en': [r'think', r'analyze', r'evaluate', r'assess', r'consider', r'judge'],
                    'fr': [r'penser', r'analyser', r'évaluer', r'considérer', r'juger'],
                    'sa': [r'man', r'cint', r'dhī']
                },
                'aspects': ['present', 'continuous', 'deliberative'],
                'semantic_fields': ['cognition', 'evaluation', 'analysis']
            },
            'FEEL': {
                'root': '√bhāv',
                'patterns': {
                    'en': [r'feel', r'emotion', r'sense', r'experience', r'perceive'],
                    'fr': [r'sentir', r'émotion', r'ressentir', r'éprouver', r'percevoir'],
                    'sa': [r'bhāv', r'ved', r'anubhū']
                },
                'aspects': ['present', 'experiential', 'continuous'],
                'semantic_fields': ['emotion', 'perception', 'experience']
            },
            'COMM': {
                'root': '√vac',
                'patterns': {
                    'en': [r'speak', r'say', r'communicate', r'tell', r'express', r'convey'],
                    'fr': [r'parler', r'dire', r'communiquer', r'exprimer', r'transmettre'],
                    'sa': [r'vac', r'bhāṣ', r'kath']
                },
                'aspects': ['present', 'past', 'continuous'],
                'semantic_fields': ['communication', 'expression', 'language']
            },
            'ITER': {
                'root': '√gam',
                'patterns': {
                    'en': [r'go', r'move', r'process', r'iterate', r'repeat', r'continue'],
                    'fr': [r'aller', r'bouger', r'processus', r'itérer', r'répéter', r'continuer'],
                    'sa': [r'gam', r'cal', r'yā']
                },
                'aspects': ['present', 'continuous', 'iterative'],
                'semantic_fields': ['movement', 'process', 'iteration']
            },
            'DECIDE': {
                'root': '√niś',
                'patterns': {
                    'en': [r'decide', r'choose', r'determine', r'resolve', r'conclude'],
                    'fr': [r'décider', r'choisir', r'déterminer', r'résoudre', r'conclure'],
                    'sa': [r'niś', r'vṛ', r'adhyavas']
                },
                'aspects': ['present', 'perfect', 'decisive'],
                'semantic_fields': ['decision', 'choice', 'resolution']
            },
            'MODAL': {
                'root': '√śak',
                'patterns': {
                    'en': [r'can', r'could', r'might', r'may', r'should', r'would', r'able'],
                    'fr': [r'peut', r'pourrait', r'devrait', r'capable', r'possible'],
                    'sa': [r'śak', r'arh', r'īś']
                },
                'aspects': ['potential', 'conditional', 'modal'],
                'semantic_fields': ['possibility', 'capability', 'modality']
            }
        }
        
        # Statistical tracking
        self.analysis_stats = {
            'documents_processed': 0,
            'hypotheses_generated': 0,
            'computation_time': 0,
            'gpu_memory_used': 0
        }
    
    def generate_hypotheses_for_document(self, document: Dict[str, Any]) -> List[DhatuHypothesis]:
        """Génère toutes les hypothèses dhātu pour un document"""
        
        content = document.get('content', '').lower()
        doc_language = document.get('language', 'en')
        
        hypotheses = []
        
        # Pour chaque dhātu
        for dhatu_name, dhatu_data in self.dhatu_database.items():
            
            # Pour chaque langue (incluant la langue du document et variantes)
            languages_to_test = [doc_language]
            if doc_language != 'en':
                languages_to_test.append('en')  # Toujours tester anglais
            if doc_language not in ['sa', 'sanskrit']:
                languages_to_test.append('sa')  # Toujours tester sanskrit
            
            for lang in languages_to_test:
                if lang not in dhatu_data['patterns']:
                    continue
                
                patterns = dhatu_data['patterns'][lang]
                
                # Pour chaque aspect
                for aspect in dhatu_data['aspects']:
                    
                    # Pour chaque pattern dans la langue
                    for pattern in patterns:
                        
                        # Match pattern dans le contenu
                        matches = re.findall(pattern, content, re.IGNORECASE)
                        
                        if matches:
                            # Calculer métriques
                            intensity = len(matches) / max(len(content.split()), 1)
                            
                            # Confidence basée sur plusieurs facteurs
                            confidence = self._calculate_confidence(
                                matches, pattern, content, lang, aspect
                            )
                            
                            # Facteurs contextuels
                            contextual_factors = self._extract_context_factors(
                                content, matches, pattern
                            )
                            
                            # Champ sémantique dominant
                            semantic_field = self._determine_semantic_field(
                                dhatu_data['semantic_fields'], content, matches
                            )
                            
                            hypothesis = DhatuHypothesis(
                                dhatu_root=dhatu_data['root'],
                                aspect=aspect,
                                intensity=intensity,
                                confidence=confidence,
                                language_variant=lang,
                                contextual_factors=contextual_factors,
                                semantic_field=semantic_field
                            )
                            
                            hypotheses.append(hypothesis)
                            self.analysis_stats['hypotheses_generated'] += 1
        
        return hypotheses
    
    def _calculate_confidence(self, matches: List[str], pattern: str, 
                            content: str, language: str, aspect: str) -> float:
        """Calcule score de confiance pour une hypothèse"""
        
        base_confidence = min(len(matches) * 0.1, 1.0)
        
        # Bonus pour correspondance de langue
        if language == 'en':
            base_confidence *= 1.1
        elif language == 'sa':
            base_confidence *= 1.3  # Sanskrit plus rare, donc plus significatif
        
        # Bonus pour aspects spéciaux
        if aspect in ['perfect', 'causal', 'deliberative']:
            base_confidence *= 1.2
        
        # Pénalité pour patterns trop communs
        if pattern in [r'\bis\b', r'\bbe\b', r'can', r'the']:
            base_confidence *= 0.7
        
        return min(base_confidence, 1.0)
    
    def _extract_context_factors(self, content: str, matches: List[str], 
                                pattern: str) -> List[str]:
        """Extrait facteurs contextuels autour des matches"""
        
        factors = []
        
        # Analyse du contexte autour de chaque match
        for match in matches[:3]:  # Limite pour performance
            match_pos = content.find(match.lower())
            if match_pos >= 0:
                # Context window
                start = max(0, match_pos - 50)
                end = min(len(content), match_pos + len(match) + 50)
                context = content[start:end]
                
                # Recherche indicateurs contextuels
                if re.search(r'\b(not|no|never)\b', context):
                    factors.append('negation')
                if re.search(r'\b(very|extremely|highly)\b', context):
                    factors.append('intensification')
                if re.search(r'\b(maybe|perhaps|possibly)\b', context):
                    factors.append('uncertainty')
                if re.search(r'\b(and|or|but)\b', context):
                    factors.append('conjunction')
        
        return list(set(factors))  # Remove duplicates
    
    def _determine_semantic_field(self, possible_fields: List[str], 
                                 content: str, matches: List[str]) -> str:
        """Détermine le champ sémantique dominant"""
        
        field_scores = {}
        
        for field in possible_fields:
            score = 0
            
            # Score basé sur mots-clés du champ
            field_keywords = {
                'connection': ['network', 'system', 'link', 'relationship'],
                'existence': ['reality', 'being', 'present', 'available'],
                'causation': ['effect', 'result', 'consequence', 'impact'],
                'cognition': ['research', 'analysis', 'study', 'investigation'],
                'emotion': ['feeling', 'sentiment', 'mood', 'attitude'],
                'communication': ['language', 'message', 'information', 'data']
            }
            
            if field in field_keywords:
                for keyword in field_keywords[field]:
                    if keyword in content.lower():
                        score += 1
            
            field_scores[field] = score
        
        # Retourne le champ avec le score le plus élevé
        return max(field_scores, key=field_scores.get) if field_scores else possible_fields[0]

# Initialisation de l'analyseur
intensive_analyzer = IntensiveDhatuAnalyzer()
print("🧬 Analyseur multi-hypothèses prêt pour traitement intensif!")

In [ ]:
# 🚀 TRAITEMENT INTENSIF MULTI-HYPOTHÈSES

def run_intensive_analysis(documents: List[Dict], batch_size: int = 10) -> Dict[str, Any]:
    """Lance l'analyse intensive multi-hypothèses"""
    
    print(f"🚀 DÉMARRAGE ANALYSE INTENSIVE")
    print(f"📊 Documents: {len(documents)}")
    print(f"🔄 Batch size: {batch_size}")
    print(f"⏱️ Estimation: {len(documents) * 2:.1f} secondes")
    print("=" * 60)
    
    start_time = time.time()
    all_results = []
    
    # Progress tracking
    total_batches = (len(documents) + batch_size - 1) // batch_size
    
    for batch_idx, i in enumerate(range(0, len(documents), batch_size)):
        batch = documents[i:i + batch_size]
        batch_start = time.time()
        
        print(f"\n📦 BATCH {batch_idx + 1}/{total_batches} ({len(batch)} docs)")
        
        batch_results = []
        
        for doc_idx, document in enumerate(batch):
            doc_start = time.time()
            
            # Génération des hypothèses pour ce document
            hypotheses = intensive_analyzer.generate_hypotheses_for_document(document)
            
            # Tri des hypothèses par confiance
            hypotheses.sort(key=lambda h: h.confidence, reverse=True)
            
            # Analyse statistique des hypothèses
            hypothesis_stats = {
                'total_hypotheses': len(hypotheses),
                'high_confidence': len([h for h in hypotheses if h.confidence > 0.7]),
                'medium_confidence': len([h for h in hypotheses if 0.3 < h.confidence <= 0.7]),
                'low_confidence': len([h for h in hypotheses if h.confidence <= 0.3]),
                'languages_detected': list(set(h.language_variant for h in hypotheses)),
                'aspects_detected': list(set(h.aspect for h in hypotheses)),
                'semantic_fields': list(set(h.semantic_field for h in hypotheses))
            }
            
            # Top hypothèses pour export
            top_hypotheses = []
            for h in hypotheses[:10]:  # Top 10
                top_hypotheses.append({
                    'dhatu_root': h.dhatu_root,
                    'aspect': h.aspect,
                    'intensity': round(h.intensity, 4),
                    'confidence': round(h.confidence, 4),
                    'language': h.language_variant,
                    'semantic_field': h.semantic_field,
                    'context_factors': h.contextual_factors
                })
            
            doc_result = {
                'document_id': document.get('id'),
                'source_file': document.get('source_file'),
                'language': document.get('language'),
                'content_length': len(document.get('content', '')),
                'processing_time_ms': round((time.time() - doc_start) * 1000, 2),
                'hypothesis_stats': hypothesis_stats,
                'top_hypotheses': top_hypotheses
            }
            
            batch_results.append(doc_result)
            intensive_analyzer.analysis_stats['documents_processed'] += 1
            
            # Progress indicator
            if (doc_idx + 1) % 5 == 0:
                elapsed = time.time() - batch_start
                rate = (doc_idx + 1) / elapsed
                print(f"   📄 {doc_idx + 1}/{len(batch)} docs | {rate:.1f} docs/sec | {hypothesis_stats['total_hypotheses']} hypotheses")
        
        batch_time = time.time() - batch_start
        all_results.extend(batch_results)
        
        print(f"   ✅ Batch terminé en {batch_time:.1f}s | {len(batch_results)} docs | {sum(r['hypothesis_stats']['total_hypotheses'] for r in batch_results)} hypotheses")
        
        # Memory cleanup pour long processing
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    total_time = time.time() - start_time
    
    # Final statistics
    final_stats = {
        'session_id': session_id,
        'timestamp': datetime.now().isoformat(),
        'processing_summary': {
            'total_documents': len(documents),
            'total_processing_time_seconds': round(total_time, 2),
            'average_time_per_document_ms': round((total_time / len(documents)) * 1000, 2),
            'documents_per_second': round(len(documents) / total_time, 2),
            'total_hypotheses_generated': intensive_analyzer.analysis_stats['hypotheses_generated'],
            'average_hypotheses_per_document': round(intensive_analyzer.analysis_stats['hypotheses_generated'] / len(documents), 1)
        },
        'gpu_info': {
            'device': str(device),
            'gpu_name': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU',
            'memory_allocated_mb': round(torch.cuda.memory_allocated() / 1e6, 2) if torch.cuda.is_available() else 0
        },
        'corpus_analysis': {
            'source_breakdown': Counter(r['source_file'] for r in all_results),
            'language_breakdown': Counter(r['language'] for r in all_results),
            'high_confidence_docs': len([r for r in all_results if r['hypothesis_stats']['high_confidence'] > 0])
        },
        'detailed_results': all_results
    }
    
    return final_stats

# LANCEMENT ANALYSE INTENSIVE
print("🧬 DÉMARRAGE TRAITEMENT INTENSIF MULTI-HYPOTHÈSES")
print("=" * 70)

if len(corpus_documents) > 0:
    # Configuration pour traitement intensif
    BATCH_SIZE = 5  # Plus petit pour monitoring détaillé
    
    intensive_results = run_intensive_analysis(corpus_documents, batch_size=BATCH_SIZE)
    
    print("\n🎯 RÉSULTATS FINAUX:")
    print(f"   📊 Documents traités: {intensive_results['processing_summary']['total_documents']}")
    print(f"   🧬 Hypothèses générées: {intensive_results['processing_summary']['total_hypotheses_generated']}")
    print(f"   ⏱️ Temps total: {intensive_results['processing_summary']['total_processing_time_seconds']}s")
    print(f"   🚀 Throughput: {intensive_results['processing_summary']['documents_per_second']} docs/sec")
    print(f"   📈 Avg hypotheses/doc: {intensive_results['processing_summary']['average_hypotheses_per_document']}")
    
else:
    print("❌ Aucun document trouvé pour le traitement")
    intensive_results = None

In [ ]:
# 💾 EXPORT RÉSULTATS et SYNCHRONISATION GITHUB

if intensive_results:
    # Sauvegarde des résultats complets
    results_filename = f"dhatu_intensive_analysis_{session_id}.json"
    results_path = f"colab_results/{results_filename}"
    
    # Créer le dossier si nécessaire
    !mkdir -p colab_results
    
    # Sauvegarde JSON complet
    with open(results_path, 'w', encoding='utf-8') as f:
        json.dump(intensive_results, f, ensure_ascii=False, indent=2, default=str)
    
    print(f"💾 Résultats sauvegardés: {results_path}")
    
    # Génération rapport Markdown exécutif
    report_filename = f"RAPPORT_INTENSIF_{session_id}.md"
    report_path = f"colab_results/{report_filename}"
    
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(f"# 🧬 Rapport Analyse Dhātu Intensive\n\n")
        f.write(f"**Session**: {session_id}\n")
        f.write(f"**Timestamp**: {intensive_results['timestamp']}\n")
        f.write(f"**GPU**: {intensive_results['gpu_info']['gpu_name']}\n\n")
        
        f.write(f"## 📊 Résumé du Traitement\n\n")
        summary = intensive_results['processing_summary']
        f.write(f"- **Documents traités**: {summary['total_documents']:,}\n")
        f.write(f"- **Hypothèses générées**: {summary['total_hypotheses_generated']:,}\n")
        f.write(f"- **Temps de traitement**: {summary['total_processing_time_seconds']}s\n")
        f.write(f"- **Throughput**: {summary['documents_per_second']} docs/sec\n")
        f.write(f"- **Moyenne hypothèses/doc**: {summary['average_hypotheses_per_document']}\n\n")
        
        f.write(f"## 🌍 Analyse du Corpus\n\n")
        corpus_analysis = intensive_results['corpus_analysis']
        f.write(f"### Sources\n")
        for source, count in corpus_analysis['source_breakdown'].items():
            f.write(f"- {source}: {count} documents\n")
        
        f.write(f"\n### Langues\n")
        for lang, count in corpus_analysis['language_breakdown'].items():
            f.write(f"- {lang}: {count} documents\n")
        
        f.write(f"\n- **Documents haute confiance**: {corpus_analysis['high_confidence_docs']}\n\n")
        
        f.write(f"## 🏆 Top Découvertes\n\n")
        
        # Analyse des meilleures hypothèses
        all_hypotheses = []
        for result in intensive_results['detailed_results']:
            for hyp in result['top_hypotheses']:
                hyp['doc_id'] = result['document_id']
                all_hypotheses.append(hyp)
        
        # Top hypothèses par confiance
        top_hyp = sorted(all_hypotheses, key=lambda x: x['confidence'], reverse=True)[:10]
        
        f.write(f"### Top 10 Hypothèses (par confiance)\n")
        for i, hyp in enumerate(top_hyp, 1):
            f.write(f"{i}. **{hyp['dhatu_root']}** ({hyp['aspect']}) - ")
            f.write(f"Confiance: {hyp['confidence']:.3f} | ")
            f.write(f"Langue: {hyp['language']} | ")
            f.write(f"Champ: {hyp['semantic_field']}\n")
        
        f.write(f"\n---\n\n")
        f.write(f"*Généré par PaniniFS Research - Analyse GPU Intensive*\n")
        f.write(f"*Fichier complet: {results_filename}*\n")
    
    print(f"📄 Rapport généré: {report_path}")
    
    # Affichage taille des fichiers
    !ls -lh colab_results/{results_filename}
    !ls -lh colab_results/{report_filename}
    
    # SYNCHRONISATION GITHUB
    print("\n🔄 Synchronisation avec GitHub...")
    
    # Add et commit
    !git add colab_results/{results_filename}
    !git add colab_results/{report_filename}
    
    commit_message = f"🧬 Analyse intensive dhātu multi-hypothèses - {session_id}"
    !git commit -m "{commit_message}"
    
    # Push vers GitHub
    !git push origin main
    
    print("✅ Résultats synchronisés avec GitHub!")
    print("\n🎯 ANALYSE INTENSIVE TERMINÉE")
    print(f"📊 {intensive_results['processing_summary']['total_hypotheses_generated']} hypothèses dhātu générées")
    print(f"🚀 Performance: {intensive_results['processing_summary']['documents_per_second']} docs/sec")
    
else:
    print("❌ Pas de résultats à exporter")